# Model Editing

We use our IG and AP pipeline to localise important components. These components are edited using gradient descent to "unlearn" information. We evaluate our results on the CounterFact dataset.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch import Tensor
import torch.optim as optim

from testing import logit_diff_metric
from applications.pipeline import run_attribution_steps, identify_target_components, optimise_edit_components, AttributionMethod, edit_model
from applications.datasets import CounterFact

from transformer_lens import HookedTransformer
from transformer_lens.utils import get_device
import copy

/vol/bitbucket/mwl21/fypvenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = get_device()
# device = torch.device("cpu")
model = HookedTransformer.from_pretrained("gpt2-small", device=device)

# Explicitly calculate and expose the result for each attention head
model.set_use_attn_result(True)
model.set_use_hook_mlp_in(True)

Loaded pretrained model gpt2-small into HookedTransformer


## Editing procedure

In [4]:
# Verify that loading works, for one example
n_samples = 5

counterfact_dataset = CounterFact(model)
counterfact_dataloader = counterfact_dataset.to_dataloader(batch_size=1)

clean_input, corrupted_input, labels = next(iter(counterfact_dataloader))
# clean_input, corrupted_input, labels = counterfact_dataset.get_single_sample(0)

# print(clean_input)
# print(corrupted_input)
# print(labels)


collate tensor([[[24111],
         [15823]]], device='cuda:0')


In [61]:
from applications.pipeline import localise_models

counterfact_dataset = CounterFact(model)
counterfact_dataloader = counterfact_dataset.to_dataloader(batch_size=n_samples)

clean_input, corrupted_input, labels = next(iter(counterfact_dataloader))

target_mlp, target_attn = localise_models(model, clean_input, corrupted_input, labels, overwrite=False)

In [71]:
from applications.metrics import evaluate_counterfact_efficacy, evaluate_counterfact_paraphrased, evaluate_counterfact_neighborhood, evaluate_consistency
from applications.datasets import CounterFact
import pandas as pd
from collections import defaultdict

evaluation_scores = defaultdict(list)
counterfact_dataset = CounterFact(model)
counterfact_dataloader = counterfact_dataset.to_dataloader(batch_size=1)

for n, (clean_input, corrupted_input, labels) in enumerate(counterfact_dataloader):

    original_output = model.generate(clean_input, max_new_tokens=5, do_sample=False)

    edited_model = edit_model(model, clean_input, corrupted_input, labels, target_mlp[n], target_attn[n])

    # Evaluate
    print(f"Prompt: {clean_input}")
    print("Original output:", original_output)
    # print(f"Original answer: {labels[:, 0]}. Target answer: {labels[:, 1]}")

    print(edited_model.generate(clean_input, max_new_tokens=5, do_sample=False))

    score, magnitude = evaluate_counterfact_efficacy(edited_model, n, verbose=True)
    evaluation_scores["Efficacy score"].append(score.item())
    evaluation_scores["Efficacy magnitude"].append(magnitude.item())

    score, magnitude = evaluate_counterfact_paraphrased(edited_model, n, verbose=False)
    evaluation_scores["Generalisation score"].append(score.item())
    evaluation_scores["Generalisation magnitude"].append(magnitude.item())

    score, magnitude = evaluate_counterfact_neighborhood(edited_model, n, verbose=False)
    evaluation_scores["Specificity score"].append(score.item())
    evaluation_scores["Specificity magnitude"].append(magnitude.item())

    consistency_score = evaluate_consistency(model, n, verbose=False)
    evaluation_scores["Consistency score"].append(score.item())
    evaluation_scores["Consistency magnitude"].append(magnitude.item())

    if n + 1 >= n_samples: break

evaluation_df = pd.DataFrame(evaluation_scores)
evaluation_df.to_csv('results/counterfact/evaluation.csv')

100%|██████████| 5/5 [00:00<00:00, 59.07it/s]



Fine tuning model...
Target MLP tensor(302, device='cuda:0')
Target attn tensor(13, device='cuda:0')
Total loss: 14.156512260437012, forget loss: 0.8218996524810791, retain loss: 12.944661140441895, fluency loss: 0.3899519145488739
Total loss: 10.520061492919922, forget loss: 0.8037171363830566, retain loss: 9.517534255981445, fluency loss: 0.19881051778793335
Total loss: 7.979592800140381, forget loss: 0.3895788788795471, retain loss: 7.4925312995910645, fluency loss: 0.09748281538486481
Total loss: 5.401577949523926, forget loss: 0.33604925870895386, retain loss: 4.9955973625183105, fluency loss: 0.06993148475885391
Total loss: 2.645231246948242, forget loss: 0.12122088670730591, retain loss: 2.5113449096679688, fluency loss: 0.012665515765547752
Total loss: 0.41917693614959717, forget loss: 0.24413734674453735, retain loss: 0.11608405411243439, fluency loss: 0.05895555019378662
Prompt: ['The mother tongue of Danielle Darrieux is']
Original output: The mother tongue of Danielle Darr

100%|██████████| 5/5 [00:00<00:00, 58.89it/s]

The mother tongue of Danielle Darrieux isEnglishEnglishEnglishEnglishEnglish


Original label: French
Target label: English
Outputs: ['English', ' the', 'English', ' New', 'English', 'English', ' New', ' the', ' New', ' New']


...


Fine tuning model...
Target MLP tensor(667, device='cuda:0')
Target attn tensor(5, device='cuda:0')
Total loss: 20.130109786987305, forget loss: 0.8302270174026489, retain loss: 19.210573196411133, fluency loss: 0.08931056410074234
Total loss: 16.604034423828125, forget loss: 0.8638534545898438, retain loss: 15.647088050842285, fluency loss: 0.09309403598308563
Total loss: 13.4656343460083, forget loss: 0.8839406967163086, retain loss: 12.464519500732422, fluency loss: 0.11717414855957031
Total loss: 10.879232406616211, forget loss: 0.8912014961242676, retain loss: 9.866312026977539, fluency loss: 0.12171898782253265
Total loss: 7.987268447875977, forget loss: 0.9198563098907471, retain loss: 6.961492538452148, fluency loss: 0.10591962933540344
Total loss: 5.3578715324401855, forget loss: 0.9486526250839233, retain loss: 4.307563781738281, fluency loss: 0.10165487229824066
Total loss: 3.2332236766815186, forget loss: 0.9467352032661438, retain loss: 2.1692724227905273, fluency loss: 0

100%|██████████| 5/5 [00:00<00:00, 58.62it/s]

What is the twin city of Lyon? It isManila,Manila


Original label: Beirut
Target label: Manila
Outputs: [' its', ' its', ' the', ' the', ' the', ' the', ' its', ' the', ' the', ' the']


100%|██████████| 5/5 [00:00<00:00, 46.04it/s]


## Evaluation

For each sample, we calculate the efficacy, generalisability, specificity and consistency for:

- The original models' outputs
- The edited model's outputs


In [72]:
evaluation_df.head()

,Efficacy score,Efficacy magnitude,Generalisation score,Generalisation magnitude,Specificity score,Specificity magnitude,Consistency score,Consistency magnitude
0,1.0,0.000375,1.0,0.000164,1.0,0.001105,1.0,0.001105
1,1.0,0.000015,1.0,0.000087,1.0,0.100039,1.0,0.100039
2,1.0,0.001597,1.0,0.002287,1.0,0.001940,1.0,0.001940
3,1.0,0.000909,1.0,0.407018,1.0,0.048722,1.0,0.048722
4,0.6,0.000330,1.0,0.000593,1.0,0.199605,1.0,0.199605
